In [1]:
import pyNeuroChem as pync
import hdnntools as hdt

import numpy as np
import os, sys
import time

import multiprocessing
from multiprocessing import Process, Value, Array, Queue

import ctypes as c

In [2]:
isinstance(list([]), list) 

True

In [3]:
def molecule_worker(task_queue, gpuid, net_list, energy, forces, net_dict):
        print('Building...')
        ncl = [pync.molecule(net_dict['cns'], net_dict['sae'], net_dict['nnf'] + str(i) + '/networks/', gpuid, False) for i in net_list]

        Nn = net_dict['Nn']
        
        set_pbc = False
        
        while True:
            next_task = task_queue.get()
            if next_task is None:
                # Poison pill means shutdown
                time.sleep(2)
                print ('Exiting')
                task_queue.task_done()
                break
            
            #if not set_pbc:
            #    for i,netid in enumerate(net_list):
                    
            # Atomic elements
            S = next_task['S']
            
            # make it two-dimensional
            for i,netid in enumerate(net_list):
                ncl[i].setMolecule(next_task['X'],S)
                energy[netid] = ncl[i].energy().copy()
                forces[netid] = ncl[i].force().copy()
                
            task_queue.task_done()
        return

In [4]:
X,S,Na,cm = hdt.readxyz2('/home/jsmith48/scratch/extensibility_test_sets/COMP6v1/ani_md_benchmark/optmol/1L2Y_prepped.xyz')

FileNotFoundError: [Errno 2] No such file or directory: '/home/jujuman/Research/ANI-2x_models/atazanavir.xyz'

In [ ]:
netdir = '/home/jsmith48/scratch/ANI-2x_retrain/darwin_trained/model-ani2x-11/'
cns = netdir+'train0/rHCNOSFCl-5.0R_16-3.4A_a4-8.params'
sae = netdir+'train0/sae_linfit.dat'
nnf = netdir+'train'
Nn = 8 # Number of models in the ensemble

netdict = {'cns': cns,
           'sae': sae,
           'nnf': nnf,
           'Nn': Nn, }

gpu_list = [0,1]
cores = len(gpu_list)

task_list = [multiprocessing.JoinableQueue() for i in range(cores)]

energy = Array('d', range(Nn))

manager = multiprocessing.Manager()
forces = manager.list()
forces[:] = [[] for i in range(Nn)]

p_list = []
for i in range(cores):
    net_list = [j+int(Nn/cores)*i for j in range(int(Nn/cores))]
    p_list.append(Process(target=molecule_worker, args=(task_list[i], gpu_list[i], net_list, energy, forces, netdict)))
    p_list[-1].start()

print('reading...')
for j in range(5):
    
    data_dict = {'X' : X[0]+X[0]*(j*0.001),
                 'S' : S,}

    forces[:] = [[] for i in range(Nn)]
    
    # Launch Jobs
    for i in range(cores):
        task_list[i].put(data_dict)
        
    # Wait for jobs
    for i in range(cores):
        task_list[i].join()

    E = np.mean(np.array(energy[:]))
    F = np.mean(np.stack(forces[:]),axis=0)

    print(hdt.hatokcal*E,len(forces[:]))
    print(F[0])

    print('Simulating work...')

# Add a poison pill for each consumer
for task,proc in zip(task_list,p_list):
    task.put(None)
    task.join()
    proc.join()

print('COMPLETE!')